In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
train=pd.read_csv("train_preprocessed.csv")

In [ ]:
train.head()

,Title,Body,Tags
0,java repeat task everi random second,alreadi familiar repeat task everi n second us...,['java']
1,java option immut,would like understand java option design immut...,['java']
2,text overlay imag darken opac react nativ,attempt overlay titl imag imag darken lower op...,"['javascript', 'image', 'react-native']"
3,ternari oper swift picki,question simpl could find answer return x hell...,"['swift', 'operators']"
4,hide show fab scale anim,use custom floatingactionmenu need implement s...,['android']


In [ ]:
train['text'] = train['Title']+' '+train['Body']

In [ ]:
train.head()

,Title,Body,Tags,text
0,java repeat task everi random second,alreadi familiar repeat task everi n second us...,['java'],java repeat task everi random second alreadi ...
1,java option immut,would like understand java option design immut...,['java'],java option immut would like understand java ...
2,text overlay imag darken opac react nativ,attempt overlay titl imag imag darken lower op...,"['javascript', 'image', 'react-native']",text overlay imag darken opac react nativ att...
3,ternari oper swift picki,question simpl could find answer return x hell...,"['swift', 'operators']",ternari oper swift picki question simpl could...
4,hide show fab scale anim,use custom floatingactionmenu need implement s...,['android'],hide show fab scale anim use custom floatinga...


In [ ]:
train.isnull().values.any()
train = train.dropna(how='any',axis=0)
train.isnull().values.any()

False

In [ ]:
train.head()

,Title,Body,Tags,text
0,java repeat task everi random second,alreadi familiar repeat task everi n second us...,['java'],java repeat task everi random second alreadi ...
1,java option immut,would like understand java option design immut...,['java'],java option immut would like understand java ...
2,text overlay imag darken opac react nativ,attempt overlay titl imag imag darken lower op...,"['javascript', 'image', 'react-native']",text overlay imag darken opac react nativ att...
3,ternari oper swift picki,question simpl could find answer return x hell...,"['swift', 'operators']",ternari oper swift picki question simpl could...
4,hide show fab scale anim,use custom floatingactionmenu need implement s...,['android'],hide show fab scale anim use custom floatinga...


In [ ]:
X=train['text']

In [ ]:
import ast
train['Tags'] = train['Tags'].apply(lambda x: ast.literal_eval(str(x)))

In [ ]:
Tags=train["Tags"]
Tags

0                                   [java]
1                                   [java]
2        [javascript, image, react-native]
3                       [swift, operators]
4                                [android]
                       ...                
43582            [c#, asp.net, sql-server]
43583                             [python]
43584                              [azure]
43585                         [ios, swift]
43586                  [c#, visual-studio]
Name: Tags, Length: 43582, dtype: object

In [ ]:
multilabel=MultiLabelBinarizer()

In [ ]:
y=multilabel.fit_transform(Tags)

In [ ]:
multilabel.classes_

array(['.htaccess', '.net', '.net-core', 'airflow', 'ajax', 'algorithm',
       'amazon-ec2', 'amazon-s3', 'amazon-web-services', 'anaconda',
       'android', 'android-activity', 'android-architecture-components',
       'android-edittext', 'android-fragments', 'android-gradle-plugin',
       'android-intent', 'android-layout', 'android-recyclerview',
       'android-room', 'android-sqlite', 'android-studio', 'angular',
       'angular-cli', 'angular-material', 'angular2-routing', 'angular5',
       'angularjs', 'animation', 'ansible', 'apache', 'apache-kafka',
       'apache-spark', 'apache-spark-sql', 'api', 'architecture',
       'arduino', 'arraylist', 'arrays', 'artificial-intelligence',
       'asp.net', 'asp.net-core', 'asp.net-core-mvc', 'asp.net-mvc',
       'asp.net-web-api', 'assembly', 'async-await', 'asynchronous',
       'audio', 'authentication', 'automation', 'awk', 'aws-api-gateway',
       'aws-lambda', 'axios', 'azure', 'azure-devops', 'babeljs', 'bash',
       'bat

In [ ]:
pd.DataFrame(y,columns=multilabel.classes_)

,.htaccess,.net,.net-core,airflow,ajax,algorithm,amazon-ec2,amazon-s3,amazon-web-services,anaconda,android,android-activity,android-architecture-components,android-edittext,android-fragments,android-gradle-plugin,android-intent,android-layout,android-recyclerview,android-room,android-sqlite,android-studio,angular,angular-cli,angular-material,angular2-routing,angular5,angularjs,animation,ansible,apache,apache-kafka,apache-spark,apache-spark-sql,api,architecture,arduino,arraylist,arrays,artificial-intelligence,...,validation,variables,vb.net,vba,vbscript,vector,video,visual-c++,visual-studio,visual-studio-2015,visual-studio-2017,visual-studio-code,vue-component,vue-router,vue.js,vuejs2,vuex,web,web-scraping,web-services,webdriver,webpack,websocket,webview,while-loop,winapi,windows,windows-10,winforms,woocommerce,wordpress,wpf,xamarin,xamarin.forms,xaml,xcode,xcode8,xcode9,xml,xpath
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43577,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43578,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43579,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
43580,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,2),max_features=50000,max_df=0.8,min_df=3,stop_words="english")
x_train=tfidf.fit_transform(x_train)

In [ ]:
x_test=tfidf.transform(x_test)

In [ ]:
len(tfidf.vocabulary_)

50000

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score,precision_score,recall_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
lr=LogisticRegression(solver='saga',C=50)
clf=OneVsRestClassifier(lr)
clf.fit(x_train,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=50, solver='saga'))

In [ ]:
predictions = clf.predict(x_test)
f1 = f1_score(y_test, predictions, average='micro')

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, predictions,average='micro')

0.3953019804412993

In [ ]:
f1

0.5666185327369422

In [ ]:
from sklearn.linear_model import SGDClassifier
lr=SGDClassifier(loss = 'hinge', alpha =0.00001)
clf=OneVsRestClassifier(lr)
clf.fit(x_train,y_train)

OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05))

In [ ]:
predictions = clf.predict(x_test)
f1 = f1_score(y_test, predictions, average='micro')
f1

0.5953808420765772

In [ ]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, predictions,average='micro')

0.42387350244943495